# DEAP
## UFAI Bootcamp: One Max Walk Through

### DEAP
DEAP is a python framework used for Genetic Programming and Genetic Algorithms
It is free, open source, and well documented

See readme for install. Install can be done with pip (recommended) or source
Python 3.4+ comes with pip, so installation can be done by from the anaconda bash

Installation example:
Open anaconda bash
Run “pip install deap”
Use Spyder IDE (run spyder in the anaconda terminal) to complete tutorial
https://github.com/deap/deap


### One Max Problem
One Max is a toy problem that takes a population of m individuals. Each individual is an n-integer vector filled with 1’s and 0’s. 
The population is allowed to evolve until an individual becomes all 1’s.

There are three tutorials for the One Max problem with deap. 
* The first is a standard introduction with no shortcuts. 
* The second is a “short version” which uses algorithms library from the deap framework. 
* The third is similar to the short version but uses numpy.

We will be walking through the most basic One Max problem example: http://deap.readthedocs.io/en/master/examples/ga_onemax.html

**NOTE** we wont be using a main since we are in a notebook

#### Import the needed libraries

In [1]:
import random
from deap import base
from deap import creator
from deap import tools

#### Creator container
* Generates a class for fitness models and individuals
* For the fitness function, a single objective (single output being optimized) is a special case of multi objectives
* Individuals inherit the fitness model class


In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

#### Toolbox
* Methods: Register (add content) and Unregister (remove content)
* Creates attributes for the integer (in this case 0 and 1)
* Creates an individual with these attributes (in this case 100)
* Creates a population of these individuals


In [5]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#### Evaluation
* One must create their own evaluation function. This should return a single value in this case because there is only a single weight in our fitness function.
* Note the comma at the end of the return, this make it return a one item list which matches weights which we initialized to one item lists (1.0,)


In [4]:
def evalOneMax(individual):
    return sum(individual),

#### Operators
* Can call a function from “tools”
* Or can lock this function into the toolbox (similar to initializations)
    * Do this one
    * The tutorial shows locking the evaluation function into the toolbox and locking premade operators into the toolbox (mate, mutate, select)


In [6]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

#### Evolution Setup
* Need to create the population which is done with the registered toolbox method population
    * In this case we are choosing to have 300 individuals (each individual is a vector of size 100)
* Evaluation is done first with the toolbox evaluate method which we made and registered into the toolbox
    * Fitnesses is a list where we applied the function evaluate to the population
    * Then we iterate through fitnesses and assign them to their appropriate individual

**NOTE** this is where the main would start

In [14]:
pop = toolbox.population(n=300)

In [15]:
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

#### Evolution Setup Continued
* Fits will hold all the fitnesses
* CXPB will be crossover probability 
* MUTPB will be mutation probability
* Then we while loop until one of the fits becomes 100 (a vector of 100 is all 1s) or we do 1000 iterations (our ending criteria)
* We create the next generation (offspring) from the population (pop) at the beginning of each loop


In [19]:
# Extracting all the fitnesses of 
fits = [ind.fitness.values[0] for ind in pop]

# Hyperparameters
CXPB = 0.1
MUTPB = 0.1

# Variable keeping track of the number of generations
g = 0

# Begin the evolution
while max(fits) < 100 and g < 1000:
    # A new generation
    g = g + 1
    print("-- Generation %i --" % g)
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    

    '''
    Mutation and crossover
    - We now need to mutate these which is done by calling functions (mate and mutate) which we locked (registered) into the toolbox (i.e. mate and mutate)
    - We want to randomly mutate individuals so CXPB and MUTPB are the chances of mutation happening
    - Then we erase (del) the old fitness value, we will recalculate next
    '''


    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
                    
                
    '''
    Evaluate
    - Very similar to original, but since we only changed a certain percent, we isolate the individuals who actually changed and only re-evaluate those
    - We also print out some statistics from the new population
    - This will run until there are only ones in a list
    '''
    
                
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    pop[:] = offspring

    
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)


-- Generation 1 --
  Min 45.0
  Max 65.0
  Avg 54.236666666666665
  Std 3.619482774590305
-- Generation 2 --
  Min 50.0
  Max 66.0
  Avg 57.663333333333334
  Std 3.168068110940108
-- Generation 3 --
  Min 53.0
  Max 66.0
  Avg 60.50666666666667
  Std 2.8029666823246555
-- Generation 4 --
  Min 57.0
  Max 68.0
  Avg 62.50333333333333
  Std 2.128377055150102
-- Generation 5 --
  Min 56.0
  Max 68.0
  Avg 63.99
  Std 1.7559138171712265
-- Generation 6 --
  Min 60.0
  Max 69.0
  Avg 65.01666666666667
  Std 1.5131828118977348
-- Generation 7 --
  Min 59.0
  Max 70.0
  Avg 65.99333333333334
  Std 1.5187568015391566
-- Generation 8 --
  Min 60.0
  Max 72.0
  Avg 67.14
  Std 1.5385274344861997
-- Generation 9 --
  Min 61.0
  Max 73.0
  Avg 68.11333333333333
  Std 1.529865644064541
-- Generation 10 --
  Min 61.0
  Max 74.0
  Avg 69.10333333333334
  Std 1.503104195397402
-- Generation 11 --
  Min 63.0
  Max 74.0
  Avg 69.97333333333333
  Std 1.6225357383498153
-- Generation 12 --
  Min 64.0
  Ma

#### My run
For me running this resulted convergence at the 213th generation
* -- Generation 213 --
  * Min 90.0
  * Max 100.0
  * Avg 98.52
  * Std 1.565119803721082
